# 🧠 Using MOISE+MARL in CybORG

This notebook demonstrates how to apply the **MOISE+MARL framework** to the **CybORG cybersecurity simulation** using role-based logic to structure defender behaviors.

## 1. Import Dependencies

In [ ]:
import gym
import math
import numpy as np
from marllib import marl
from mma_wrapper.label_manager import label_manager
from mma_wrapper.organizational_model import (
    organizational_model, structural_specifications,
    functional_specifications, deontic_specifications,
    deontic_specification, time_constraint_type
)
from mma_wrapper.organizational_specification_logic import role_logic
from mma_wrapper.utils import label, observation, action, trajectory
from marllib.envs.base_env.cyborg import create_env
from CybORG.Simulator.Actions.ConcreteActions.RemoveOtherSessions import RemoveOtherSessions
from CybORG.Simulator.Actions.ConcreteActions.ControlTraffic import BlockTraffic
from CybORG.Simulator.Actions.ConcreteActions.ExploitActions.RetakeControl import RetakeControl
from CybORG.Simulator.Actions import Sleep
from ipaddress import IPv4Address

## 2. Define the Label Manager

In [ ]:
class cyborg_label_manager(label_manager):
    def __init__(self, ip_list, ip_host_map, msg_len, agent_host_map, int_to_action, action_space=None, observation_space=None):
        super().__init__(action_space, observation_space)
        self.ip_list = ip_list
        self.ip_host_map = ip_host_map
        self.msg_len = msg_len
        self.agent_host_map = agent_host_map
        self.int_to_action = int_to_action

    def get_ip_from_host(self, hostname):
        for ip, host in self.ip_host_map.items():
            if host == hostname:
                return str(ip)
        return None

    def get_host_from_agent(self, agent_name):
        return self.agent_host_map.get(agent_name, None)

    def one_hot_decode_observation(self, observation, agent=None):
        num_drones = len(self.ip_list)
        obs = {}
        idx = 0
        success_val = observation[idx]
        success_enum = {0: "FAILURE", 1: "SUCCESS", 2: "UNKNOWN"}
        obs["success"] = success_enum.get(success_val + 1, "UNKNOWN")
        idx += 1
        own_drone = self.agent_host_map[agent]
        own_ip = next(ip for ip, host in self.ip_host_map.items() if host == own_drone)
        obs[own_drone] = {
            "Interface": [{"Interface Name": "wlan0", "IP Address": str(own_ip)}],
            "System info": {},
        }
        blocked_ips = [str(ip) for i, ip in enumerate(self.ip_list) if observation[idx + i] == 1]
        idx += num_drones
        obs[own_drone]["Interface"][0]["blocked_ips"] = blocked_ips
        if observation[idx] == 1:
            obs[own_drone]["Processes"] = [{"PID": 1094, "Username": "root"}]
        idx += 1
        net_conns = [{"remote_address": str(ip)} for i, ip in enumerate(self.ip_list) if observation[idx + i] == 1]
        idx += num_drones
        if net_conns:
            obs[own_drone]["Interface"][0]["NetworkConnections"] = net_conns
        obs[own_drone]["System info"]["position"] = [observation[idx], observation[idx + 1]]
        idx += 2
        obs.update({host: {"System info": {}} for ip, host in self.ip_host_map.items() if host != own_drone})
        for _ in range(num_drones - 1):
            drone_id = observation[idx]; idx += 1
            pos_x = observation[idx]; pos_y = observation[idx + 1]; idx += 2
            has_session = observation[idx] == 1; idx += 1
            ip = self.ip_list[drone_id]
            hostname = self.ip_host_map[ip]
            obs[hostname]["System info"]["position"] = [pos_x, pos_y]
            if has_session:
                obs[hostname]["Sessions"] = [{"Username": "root", "ID": 0}]
        if self.msg_len > 0:
            obs["message"] = observation[idx:idx + self.msg_len].tolist()
        return obs

    def one_hot_encode_action(self, action, agent=None):
        for idx, cyborg_action in self.int_to_action[agent].items():
            if type(cyborg_action) == type(action):
                if cyborg_action.__dict__ == {k: IPv4Address(v) if k == "ip_address" and isinstance(v, str) else v for k, v in action.__dict__.items()}:
                    return idx
        raise ValueError("Action not found")

    def one_hot_decode_action(self, action, agent=None):
        return self.int_to_action[agent][action]

## 3. Define Role Logic Functions

In [ ]:
def primary_fun_system_cleaner(traj, obs, agent_name, label_mgr):
    data = label_mgr.one_hot_decode_observation(obs, agent=agent_name)
    hostname = label_mgr.get_host_from_agent(agent_name)
    proc = 'Processes' in data.get(hostname, {})
    conn = any('NetworkConnections' in iface and iface['NetworkConnections']
               for iface in data.get(hostname, {}).get('Interface', []))
    if proc or conn:
        return label_mgr.one_hot_encode_action(RemoveOtherSessions(agent=agent_name, session=0), agent_name)
    return label_mgr.one_hot_encode_action(Sleep(), agent_name)

def primary_fun_firewall_operator(traj, obs, agent_name, label_mgr):
    data = label_mgr.one_hot_decode_observation(obs, agent=agent_name)
    hostname = label_mgr.get_host_from_agent(agent_name)
    seen = []
    for iface in data.get(hostname, {}).get("Interface", []):
        for conn in iface.get("NetworkConnections", []):
            ip = conn.get("remote_address")
            if ip and ip not in iface.get("blocked_ips", []):
                seen.append(ip)
    if seen:
        return label_mgr.one_hot_encode_action(BlockTraffic(agent=agent_name, ip_address=seen[0], session=0), agent_name)
    return label_mgr.one_hot_encode_action(Sleep(), agent_name)

def primary_fun_system_rescuer(traj, obs, agent_name, label_mgr):
    data = label_mgr.one_hot_decode_observation(obs, agent=agent_name)
    hostname = label_mgr.get_host_from_agent(agent_name)
    if "Sessions" not in data.get(hostname, {}):
        ip = label_mgr.get_ip_from_host(hostname)
        return label_mgr.one_hot_encode_action(RetakeControl(agent=agent_name, ip_address=ip, session=0), agent_name)
    return label_mgr.one_hot_encode_action(Sleep(), agent_name)

## 4. Setup Environment and Organizational Model

In [ ]:
_env = create_env()
_env.reset()
ip_list = _env.ip_addresses
ip_host_map = {ip: host for host, ip in _env.env.get_ip_map().items()}
msg_len = _env.msg_len
agent_host_map = _env.agent_host_map
int_to_action = _env.int_to_action

label_mgr = cyborg_label_manager(ip_list, ip_host_map, msg_len, agent_host_map, int_to_action)

cyborg_model = organizational_model(
    structural_specifications(
        roles={
            "SystemCleaner": role_logic(label_manager=label_mgr).registrer_script_rule(primary_fun_system_cleaner),
            "FirewallOperator": role_logic(label_manager=label_mgr).registrer_script_rule(primary_fun_firewall_operator),
            "SystemRescuer": role_logic(label_manager=label_mgr).registrer_script_rule(primary_fun_system_rescuer)
        },
        role_inheritance_relations={}, root_groups={}
    ),
    functional_specifications=functional_specifications(goals={}, social_scheme={}, mission_preferences=[]),
    deontic_specifications=deontic_specifications(
        obligations=[
            deontic_specification("SystemCleaner", ["blue_agent_0"], [], time_constraint_type.ANY),
            deontic_specification("FirewallOperator", ["blue_agent_1"], [], time_constraint_type.ANY),
            deontic_specification("SystemRescuer", ["blue_agent_2"], [], time_constraint_type.ANY),
        ],
        permissions=[]
    )
)

## 5. Create Environment with MOISE+MARL

In [ ]:
env = marl.make_env(environment_name="cyborg", map_name="cage3", organizational_model=cyborg_model)

## 6. Initialize Algorithm and Render

In [ ]:
mappo = marl.algos.mappo(hyperparam_source="test")
model = marl.build_model(env, mappo, {"core_arch": "mlp", "encode_layer": "128-256"})

# mappo.fit(env, model, stop={'timesteps_total': 2e6})  # Optional training

mappo.render(env, model,
    restore_path={
        'params_path': "./exp_results/mappo_mlp_cage3_copy/.../params.json",
        'model_path': "./exp_results/mappo_mlp_cage3_copy/.../checkpoint-20",
        'render_env': True
    },
    local_mode=True,
    share_policy="group"
)

## ✅ Conclusion
In this notebook, we:
- Defined organizational roles for blue agents in CybORG
- Mapped each role to script-based policies
- Created a MOISE+MARL organizational model
- Visualized and validated behavior using the rendering engine.